In [7]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader, random_split


In [8]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.x1 = df.iloc[:, 0].values
        self.x2 = df.iloc[:, 1].values
        self.x3 = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.length = len(df)

    def __getitem__(self, index):
        x = torch.FloatTensor([self.x1[index], self.x2[index], self.x3[index]])#index는 개발자가 직접 지정하는 것이 아니라 DataLoader가 매 반복마다 알아서 넘겨주는 값입니다.
        y = torch.FloatTensor([int(self.y[index])])
        return x, y

    def __len__(self):
        return self.length#self 없이 선언하면 그 메서드 안에서만 존재하는 지역 변수가 되어 다른 메서드에서 사용할 수 없습니다.
    #self는 **"이 객체 자신"**을 가리키는 것이고, self.x는 **"이 객체에 x라는 데이터를 붙여둔다"**는 의미입니다 self.x 에서 x가 변수면 self.x 는 객체 변수그ㅓㅓㅓ.

#### __getitem__ 이호출되면
이런 값이 나온다  return (tensor([1.2, 0.7, 1.1]), tensor([0.0])) 앞은 특성(입력) 뒤는 레이블 

In [9]:
#입력 3개(x1, x2, x3)를 받아서 0~1 사이의 확률값 1개를 출력하는 이진 분류 모델입니다.
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
          nn.Linear(3, 1), #nn.Linear는 입력 데이터에 가중치를 곱하고 편향을 더하는 연산을 하며, 이 가중치와 편향이 학습 과정에서 자동으로 최적화됩니다.
          nn.Sigmoid()
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [10]:
# model = CustomModel()
# print(list(model.parameters()))

#### nn.Sequential이란
#### 여러 연산을 순서대로 실행해주는 컨테이너입니다.
####   self.layer = nn.Sequential(
####   nn.Linear(3, 1),   # 1번째 실행
####   nn.Sigmoid()        # 2번째 실행
####  )

In [15]:
dataset = CustomDataset("/Users/macminim4/PyCharmMiscProject/Aiffel/feb/pytorch/data/datasets/binary.csv")
dataset_size = len(dataset)
train_size = int(dataset_size * 0.8)
validation_size = int(dataset_size * 0.1)
test_size = dataset_size - train_size - validation_size

train_dataset, validation_dataset, test_dataset = random_split(dataset, [train_size, validation_size, test_size], torch.manual_seed(4))
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True, drop_last=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=4, shuffle=True, drop_last=True)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=True, drop_last=True)

In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.BCELoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [13]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch+1:4d}, Model : {list(model.parameters())}, Cost : {cost:.3f}")

Epoch : 1000, Model : [Parameter containing:
tensor([[-0.3594, -0.2363,  0.5563]], requires_grad=True), Parameter containing:
tensor([0.4447], requires_grad=True)], Cost : 0.714
Epoch : 2000, Model : [Parameter containing:
tensor([[-0.3217, -0.2004,  0.5950]], requires_grad=True), Parameter containing:
tensor([0.4314], requires_grad=True)], Cost : 0.651
Epoch : 3000, Model : [Parameter containing:
tensor([[-0.2970, -0.1746,  0.6208]], requires_grad=True), Parameter containing:
tensor([0.4143], requires_grad=True)], Cost : 0.662
Epoch : 4000, Model : [Parameter containing:
tensor([[-0.2800, -0.1550,  0.6389]], requires_grad=True), Parameter containing:
tensor([0.3948], requires_grad=True)], Cost : 0.631
Epoch : 5000, Model : [Parameter containing:
tensor([[-0.2678, -0.1391,  0.6522]], requires_grad=True), Parameter containing:
tensor([0.3740], requires_grad=True)], Cost : 0.661
Epoch : 6000, Model : [Parameter containing:
tensor([[-0.2588, -0.1258,  0.6623]], requires_grad=True), Parame

In [14]:
with torch.no_grad():
    model.eval()
    for x, y in validation_dataloader:
        x = x.to(device)
        y = y.to(device)
        
        outputs = model(x)

        print(outputs)
        print(outputs >= torch.FloatTensor([0.5]).to(device))
        print("--------------------")

tensor([[0.8500],
        [0.6789],
        [0.6233],
        [0.6530]])
tensor([[True],
        [True],
        [True],
        [True]])
--------------------
tensor([[0.8203],
        [0.8698],
        [0.8618],
        [0.6455]])
tensor([[True],
        [True],
        [True],
        [True]])
--------------------
